In [15]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 200)
%matplotlib inline

# 训练集

## 4G训练集

In [16]:
train4 = pd.read_csv('train_pre.csv')

In [17]:
train4['gender'] = train4['gender'].astype(int)
train4['age'] = train4['age'].astype(int)

In [18]:
train4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 743980 entries, 0 to 743979
Data columns (total 27 columns):
service_type              743980 non-null int64
is_mix_service            743980 non-null int64
online_time               743980 non-null int64
1_total_fee               743980 non-null float64
2_total_fee               743980 non-null float64
3_total_fee               743980 non-null float64
4_total_fee               743980 non-null float64
month_traffic             743980 non-null float64
many_over_bill            743980 non-null int64
contract_type             743980 non-null int64
contract_time             743980 non-null int64
is_promise_low_consume    743980 non-null int64
net_service               743980 non-null int64
pay_times                 743980 non-null int64
pay_num                   743980 non-null float64
last_month_traffic        743980 non-null float64
local_trafffic_month      743980 non-null float64
local_caller_time         743980 non-null float64
service

In [19]:
train4_id = train4[['user_id']]
train4_y = train4[['current_service']]
train4_x = train4.drop(['user_id', 'current_service'], axis=1)
train4_feature = train4_x.columns

In [20]:
from sklearn.preprocessing import LabelEncoder
le4 = LabelEncoder()
train4_y_le = le4.fit_transform(train4_y['current_service'])

In [21]:
# from sklearn.decomposition import PCA

# pca = PCA()
# train4_x_stand = pca.fit_transform(train4_x)

train4_x_stand = train4_x

In [22]:
train4_tra_x = train4_x_stand[1:int(4/5*len(train4_x_stand))]
train4_val_x = train4_x_stand[int(4/5*len(train4_x_stand)):]

In [23]:
train4_tra_y = train4_y_le[1:int(4/5*len(train4_x_stand))]
train4_val_y = train4_y_le[int(4/5*len(train4_x_stand)):]

lgb可以不用one-hot编码，但是要指出类别特征的名字

In [28]:
column_str = ['service_type', 'is_mix_service', 'many_over_bill', 'contract_type', 'is_promise_low_consume', 'net_service', 'gender']
train_data4 = lgb.Dataset(train4_tra_x, label=train4_tra_y, feature_name=list(train4_tra_x.columns), categorical_feature=column_str)
validate_data4 = lgb.Dataset(train4_val_x, train4_val_y, feature_name=list(train4_tra_x.columns), categorical_feature=column_str)
train_all4 = lgb.Dataset(train4_x_stand, train4_y_le, feature_name=list(train4_tra_x.columns), categorical_feature=column_str)

learning_rate: .05 n_estimators:1701

In [25]:
params4 = {'learning_rate':0.05,
          'boosting_type': 'gbdt', 
           'min_split_gain':0.1, 
          'lambda_l1':0.1,
          'lambda_l2':0.2,
          'max_depth':8,
          'num_leaves':150, 
#            'n_estimators':3000, 
          'objective':'multiclass',
          'num_class':11
          }

max_depth=7 

base_fee : 0.1572 0.703down

user_num : 0.157363 0.722 down

fee_mean var etc :

baseline : 0.1576

In [ ]:
cv_result = lgb.cv(params4, train_all4, num_boost_round=3000, nfold=5, verbose_eval=True)

[1]	cv_agg's multi_logloss: 2.18091 + 0.000390432
[2]	cv_agg's multi_logloss: 2.01263 + 0.000509203
[3]	cv_agg's multi_logloss: 1.87465 + 0.000680646
[4]	cv_agg's multi_logloss: 1.75723 + 0.000879295
[5]	cv_agg's multi_logloss: 1.65507 + 0.00103145
[6]	cv_agg's multi_logloss: 1.56473 + 0.00111618
[7]	cv_agg's multi_logloss: 1.48379 + 0.00121249
[8]	cv_agg's multi_logloss: 1.41099 + 0.0014832
[9]	cv_agg's multi_logloss: 1.34451 + 0.00139442
[10]	cv_agg's multi_logloss: 1.28365 + 0.00145864
[11]	cv_agg's multi_logloss: 1.22766 + 0.00142829
[12]	cv_agg's multi_logloss: 1.17589 + 0.0015677
[13]	cv_agg's multi_logloss: 1.12781 + 0.00148896
[14]	cv_agg's multi_logloss: 1.08306 + 0.00147912
[15]	cv_agg's multi_logloss: 1.04128 + 0.00154844
[16]	cv_agg's multi_logloss: 1.00211 + 0.00167017
[17]	cv_agg's multi_logloss: 0.965314 + 0.00151982
[18]	cv_agg's multi_logloss: 0.930855 + 0.00147298
[19]	cv_agg's multi_logloss: 0.898364 + 0.00147542
[20]	cv_agg's multi_logloss: 0.867665 + 0.00155867
[21

[162]	cv_agg's multi_logloss: 0.22078 + 0.00190628
[163]	cv_agg's multi_logloss: 0.220443 + 0.00192042
[164]	cv_agg's multi_logloss: 0.220133 + 0.00192369
[165]	cv_agg's multi_logloss: 0.219822 + 0.00193919
[166]	cv_agg's multi_logloss: 0.219515 + 0.00193194
[167]	cv_agg's multi_logloss: 0.219214 + 0.00193668
[168]	cv_agg's multi_logloss: 0.21891 + 0.00193196
[169]	cv_agg's multi_logloss: 0.218605 + 0.00192895
[170]	cv_agg's multi_logloss: 0.218304 + 0.00193845
[171]	cv_agg's multi_logloss: 0.218003 + 0.00194148
[172]	cv_agg's multi_logloss: 0.217717 + 0.00194827
[173]	cv_agg's multi_logloss: 0.217418 + 0.00195258
[174]	cv_agg's multi_logloss: 0.217139 + 0.00196677
[175]	cv_agg's multi_logloss: 0.216833 + 0.00196727
[176]	cv_agg's multi_logloss: 0.216539 + 0.00198943
[177]	cv_agg's multi_logloss: 0.216252 + 0.00199638
[178]	cv_agg's multi_logloss: 0.215955 + 0.00200168
[179]	cv_agg's multi_logloss: 0.215653 + 0.00201091
[180]	cv_agg's multi_logloss: 0.215377 + 0.00201181
[181]	cv_agg's

[321]	cv_agg's multi_logloss: 0.192744 + 0.00204826
[322]	cv_agg's multi_logloss: 0.192659 + 0.00203711
[323]	cv_agg's multi_logloss: 0.192561 + 0.00203587
[324]	cv_agg's multi_logloss: 0.192466 + 0.00203527
[325]	cv_agg's multi_logloss: 0.192358 + 0.00204403
[326]	cv_agg's multi_logloss: 0.192271 + 0.00205051
[327]	cv_agg's multi_logloss: 0.192172 + 0.00204865
[328]	cv_agg's multi_logloss: 0.192072 + 0.00205588
[329]	cv_agg's multi_logloss: 0.191979 + 0.00205295
[330]	cv_agg's multi_logloss: 0.191894 + 0.00206709
[331]	cv_agg's multi_logloss: 0.19181 + 0.00207263
[332]	cv_agg's multi_logloss: 0.191718 + 0.00208039
[333]	cv_agg's multi_logloss: 0.191619 + 0.00208915
[334]	cv_agg's multi_logloss: 0.191533 + 0.00208256
[335]	cv_agg's multi_logloss: 0.191452 + 0.00208443
[336]	cv_agg's multi_logloss: 0.191366 + 0.00208281
[337]	cv_agg's multi_logloss: 0.191273 + 0.00208387
[338]	cv_agg's multi_logloss: 0.191197 + 0.00207874
[339]	cv_agg's multi_logloss: 0.191112 + 0.00207542
[340]	cv_agg'

[480]	cv_agg's multi_logloss: 0.182153 + 0.00212578
[481]	cv_agg's multi_logloss: 0.1821 + 0.00213811
[482]	cv_agg's multi_logloss: 0.182051 + 0.00213901
[483]	cv_agg's multi_logloss: 0.181997 + 0.00214721
[484]	cv_agg's multi_logloss: 0.181945 + 0.00214745
[485]	cv_agg's multi_logloss: 0.181895 + 0.00214457
[486]	cv_agg's multi_logloss: 0.181836 + 0.00213739
[487]	cv_agg's multi_logloss: 0.181784 + 0.00213757
[488]	cv_agg's multi_logloss: 0.181737 + 0.0021376
[489]	cv_agg's multi_logloss: 0.181681 + 0.00214396
[490]	cv_agg's multi_logloss: 0.18164 + 0.00215309
[491]	cv_agg's multi_logloss: 0.181593 + 0.00214557
[492]	cv_agg's multi_logloss: 0.181548 + 0.00215252
[493]	cv_agg's multi_logloss: 0.181507 + 0.00214856
[494]	cv_agg's multi_logloss: 0.181462 + 0.00214805
[495]	cv_agg's multi_logloss: 0.181411 + 0.00215252
[496]	cv_agg's multi_logloss: 0.181359 + 0.00216012
[497]	cv_agg's multi_logloss: 0.181313 + 0.00215015
[498]	cv_agg's multi_logloss: 0.181268 + 0.00216333
[499]	cv_agg's m

[639]	cv_agg's multi_logloss: 0.176007 + 0.00220523
[640]	cv_agg's multi_logloss: 0.175976 + 0.0022035
[641]	cv_agg's multi_logloss: 0.175949 + 0.00220383
[642]	cv_agg's multi_logloss: 0.175919 + 0.00220048
[643]	cv_agg's multi_logloss: 0.175881 + 0.00219448
[644]	cv_agg's multi_logloss: 0.175852 + 0.00219703
[645]	cv_agg's multi_logloss: 0.175823 + 0.0022005
[646]	cv_agg's multi_logloss: 0.175789 + 0.00220179
[647]	cv_agg's multi_logloss: 0.175769 + 0.00220204
[648]	cv_agg's multi_logloss: 0.175737 + 0.00219562
[649]	cv_agg's multi_logloss: 0.175705 + 0.00219556
[650]	cv_agg's multi_logloss: 0.175671 + 0.00219948
[651]	cv_agg's multi_logloss: 0.175635 + 0.00219832
[652]	cv_agg's multi_logloss: 0.175608 + 0.00219758
[653]	cv_agg's multi_logloss: 0.175578 + 0.00220417
[654]	cv_agg's multi_logloss: 0.175549 + 0.00220787
[655]	cv_agg's multi_logloss: 0.175523 + 0.0022058
[656]	cv_agg's multi_logloss: 0.175491 + 0.00220883
[657]	cv_agg's multi_logloss: 0.175458 + 0.00221267
[658]	cv_agg's 

...

[2199]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2200]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2201]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2202]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2203]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2204]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2205]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2206]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2207]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2208]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2209]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2210]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2211]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2212]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2213]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2214]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2215]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2216]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2217]	cv_agg's multi_logloss: 0.164805 + 0.00

[2354]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2355]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2356]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2357]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2358]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2359]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2360]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2361]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2362]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2363]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2364]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2365]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2366]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2367]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2368]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2369]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2370]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2371]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2372]	cv_agg's multi_logloss: 0.164805 + 0.00

[2509]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2510]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2511]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2512]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2513]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2514]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2515]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2516]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2517]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2518]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2519]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2520]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2521]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2522]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2523]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2524]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2525]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2526]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2527]	cv_agg's multi_logloss: 0.164805 + 0.00

[2664]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2665]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2666]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2667]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2668]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2669]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2670]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2671]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2672]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2673]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2674]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2675]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2676]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2677]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2678]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2679]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2680]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2681]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2682]	cv_agg's multi_logloss: 0.164805 + 0.00

[2819]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2820]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2821]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2822]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2823]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2824]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2825]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2826]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2827]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2828]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2829]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2830]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2831]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2832]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2833]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2834]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2835]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2836]	cv_agg's multi_logloss: 0.164805 + 0.00203005
[2837]	cv_agg's multi_logloss: 0.164805 + 0.00

0.16202

In [12]:
lgb4 = lgb.train(params4, train_all4)

# 测试集

## 4G测试集

In [13]:
test4 = pd.read_csv('test4_featured.csv')

In [14]:
test4_id = test4[['user_id']]
test4_x = test4.drop(['user_id'], axis=1)

In [15]:
test4_x_stand = test4_x

In [16]:
pre_test4 = lgb4.predict(test4_x)

In [209]:
pre_test4 = [list(x).index(max(x)) for x in pre_test4]

In [210]:
pre_test4 = le4.inverse_transform(pre_test4)

In [211]:
predict4 = test4_id
predict4['predict_service'] = pre_test4

In [212]:
predict4.to_csv('lgb_all108_7_100.csv', index=None)

In [182]:
predict4.head()

,user_id,predict_service
0,012pSke7DsCrR985,90063345
1,012re3ZJSB6FptHW,90063345
2,013ElTBrRntuZDXz,90063345
3,0146B9qQbNxIynoX,90063345
4,0149vWf6u8UrKyJT,90063345


In [320]:
# train_data = lgb.Dataset(train4_x_stand, label=train4_y_le)

In [284]:
# params = {'learning_rate':0.05,
#           'boosting': 'gbdt', 
#           'lambda_l1':0.1,
#           'lambda_l2':0.2,
#           'objective':'multiclass',
#           'n_estimators':10000, 
#           'num_class':11}

In [285]:
# best_params = {}
# for num_leaves in range(40,200,5):
#     for max_depth in range(3,8,1):
#         params['num_leaves'] = num_leaves
#         params['max_depth'] = max_depth

#         cv_results = lgb.cv(
#                             params,
#                             train_data,
#                             nfold=3,
#                             metrics='multi_logloss',
#                             early_stopping_rounds=10,
#                             verbose_eval=True
#                             )

#         mean_merror = pd.Series(cv_results['multi_logloss-mean']).min()
#         boost_rounds = pd.Series(cv_results['multi_logloss-mean']).argmin()

#         if mean_merror < min_merror:
#             min_merror = mean_merror
#             best_params['num_leaves'] = num_leaves
#             best_params['max_depth'] = max_depth

# params['num_leaves'] = best_params['num_leaves']
# params['max_depth'] = best_params['max_depth']

In [53]:
import matplotlib.pyplot as plt

print(pd.DataFrame({
        'column': train4_feature,
        'importance': lgb4.feature_importance(),
    }).sort_values(by='importance'))

# plt.plot(sorted(lgb4.feature_importance()))

                    column  importance
23    former_complaint_num        2278
0             service_type        2659
22         complaint_level        3722
12             net_service        5313
1           is_mix_service        5845
24    former_complaint_fee        6341
11  is_promise_low_consume        9210
8           many_over_bill       12537
20                  gender       17555
13               pay_times       17595
9            contract_type       19876
10           contract_time       24150
18    service1_caller_time       40377
17       local_caller_time       50570
21                     age       52217
15      last_month_traffic       54150
16    local_trafffic_month       61580
14                 pay_num       61681
4              2_total_fee       74170
7            month_traffic       75055
6              4_total_fee       76570
2              online_time       79548
5              3_total_fee       81915
19    service2_caller_time       85719
3              1_total_fe

In [72]:
val_lgb4 = lgb4.predict(train4_val_x)
val_lgb4 = [list(x).index(max(x)) for x in val_lgb4]

KeyboardInterrupt: 

In [50]:
validation4 = train4_id[int(4/5*len(train4_x_stand)):]
validation4['predict_service'] = le4.inverse_transform(val_lgb4)

In [51]:
val_y4 = train4_id[int(4/5*len(train4_x_stand)):]
val_y4['current_service'] = le4.inverse_transform(train4_val_y)

use_situation 上升

pay caller_time 都没什么用 age_like会降分

total_fee: 0.84129756 0.7347

online_mix:0.83459 0.7340

baseline_7_100: 0.829 线上0.738

In [52]:
score(val_y4, validation4)

0.9566072526121696 99999825
0.8960544217687075 89950167
0.9666816749212067 89950168
0.9704851294809453 90155946
0.9974457104499211 90063345
0.8460517584605176 99999826
0.9386717154842782 89950166
0.8534657283331726 99999828
0.8095888332574723 99999827
0.9958054790929348 90109916
0.7875480590350985 99999830


0.8294913556201344

## 2I2C训练集

In [152]:
train1 = pd.read_csv('train1.csv')

In [153]:
train1_id = train1[['user_id']]
train1_y = train1[['current_service']]
train1_x = train1.drop(['service_type', 'gender', 'user_id', 'current_service'], axis=1)

In [154]:
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
train1_y_le = le1.fit_transform(train1_y['current_service'])

In [155]:
train1_x_stand = train1_x

In [156]:
train1_tra_x = train1_x_stand[1:int(4/5*len(train1_x_stand))]
train1_val_x = train1_x_stand[int(4/5*len(train1_x_stand)):]
train1_tra_y = train1_y_le[1:int(4/5*len(train1_x_stand))]
train1_val_y = train1_y_le[int(4/5*len(train1_x_stand)):]

In [157]:
train_data1 = lgb.Dataset(train1_tra_x, label=train1_tra_y)
validate_data1 = lgb.Dataset(train1_val_x, train1_val_y)

In [158]:
params1 = {'learning_rate':0.05,
          'boosting': 'gbdt', 
          'lambda_l1':0.1,
          'lambda_l2':0.2,
          'max_depth':4,
          'num_leaves':200, 
           'n_estimators':400, 
          'objective':'multiclass',
          'num_class':11}

In [159]:
lgb1 = lgb.train(params1, train_data1, valid_sets=[validate_data1])

[1]	valid_0's multi_logloss: 0.912949
[2]	valid_0's multi_logloss: 0.857502
[3]	valid_0's multi_logloss: 0.806441
[4]	valid_0's multi_logloss: 0.759296
[5]	valid_0's multi_logloss: 0.715643
[6]	valid_0's multi_logloss: 0.675045
[7]	valid_0's multi_logloss: 0.637369
[8]	valid_0's multi_logloss: 0.602149
[9]	valid_0's multi_logloss: 0.569233
[10]	valid_0's multi_logloss: 0.538474
[11]	valid_0's multi_logloss: 0.50974
[12]	valid_0's multi_logloss: 0.482793
[13]	valid_0's multi_logloss: 0.45754
[14]	valid_0's multi_logloss: 0.433829
[15]	valid_0's multi_logloss: 0.411556
[16]	valid_0's multi_logloss: 0.390572
[17]	valid_0's multi_logloss: 0.370888
[18]	valid_0's multi_logloss: 0.352335
[19]	valid_0's multi_logloss: 0.334867
[20]	valid_0's multi_logloss: 0.318393
[21]	valid_0's multi_logloss: 0.302864
[22]	valid_0's multi_logloss: 0.288176
[23]	valid_0's multi_logloss: 0.274332
[24]	valid_0's multi_logloss: 0.261243
[25]	valid_0's multi_logloss: 0.248882
[26]	valid_0's multi_logloss: 0.2372

[209]	valid_0's multi_logloss: 0.0130328
[210]	valid_0's multi_logloss: 0.0130212
[211]	valid_0's multi_logloss: 0.0130023
[212]	valid_0's multi_logloss: 0.0129977
[213]	valid_0's multi_logloss: 0.0129846
[214]	valid_0's multi_logloss: 0.0129797
[215]	valid_0's multi_logloss: 0.0129709
[216]	valid_0's multi_logloss: 0.0129676
[217]	valid_0's multi_logloss: 0.0129537
[218]	valid_0's multi_logloss: 0.0129416
[219]	valid_0's multi_logloss: 0.0129248
[220]	valid_0's multi_logloss: 0.0129195
[221]	valid_0's multi_logloss: 0.012905
[222]	valid_0's multi_logloss: 0.0129011
[223]	valid_0's multi_logloss: 0.0128925
[224]	valid_0's multi_logloss: 0.0128813
[225]	valid_0's multi_logloss: 0.0128757
[226]	valid_0's multi_logloss: 0.0128616
[227]	valid_0's multi_logloss: 0.0128526
[228]	valid_0's multi_logloss: 0.0128507
[229]	valid_0's multi_logloss: 0.0128426
[230]	valid_0's multi_logloss: 0.0128356
[231]	valid_0's multi_logloss: 0.0128251
[232]	valid_0's multi_logloss: 0.0128187
[233]	valid_0's m

In [160]:
val_lgb1 = lgb1.predict(train1_val_x)
val_lgb1 = [list(x).index(max(x)) for x in val_lgb1]

In [161]:
validation1 = train1_id[int(4/5*len(train1_x_stand)):]
validation1['predict_service'] = le1.inverse_transform(val_lgb1)

In [162]:
val_y1 = train1_id[int(4/5*len(train1_x_stand)):]
val_y1['current_service'] = le1.inverse_transform(train1_val_y)

baseline: 0.977

In [163]:
score(val_y1, validation1)

0.9975177891775608 90063345
0.9718612272573172 90155946
0.9960722702278083 90109916


0.9771001481743756

In [164]:
import matplotlib.pyplot as plt

train1_feature = train1_x.columns
print(pd.DataFrame({
        'column': train1_feature,
        'importance': lgb1.feature_importance(),
    }).sort_values(by='importance'))

                    column  importance
0           is_mix_service           0
24       little_online_mix           0
23         many_online_mix           0
25      many_online_notmix           0
10  is_promise_low_consume           0
9            contract_time           0
8            contract_type           0
26    little_online_notmix           0
21    former_complaint_num          44
22    former_complaint_fee         207
20         complaint_level         303
11             net_service         697
7           many_over_bill        2519
12               pay_times        3066
14      last_month_traffic        3526
17    service1_caller_time        5909
16       local_caller_time        6111
13                 pay_num        8128
19                     age        9269
1              online_time        9294
6            month_traffic       10064
18    service2_caller_time       12370
15    local_trafffic_month       12498
3              2_total_fee       12936
5              4_total_fe

## 2I2C测试集

In [173]:
test1_id = test1[['user_id']]
test1_x = test1.drop(['service_type','user_id'], axis=1)

In [174]:
test1_x_stand = test1_x

In [175]:
pre_test1 = lgb1.predict(test1_x_stand)

In [176]:
pre_test1 = [list(x).index(max(x)) for x in pre_test1]

In [177]:
pre_test1 = le1.inverse_transform(pre_test1)

In [178]:
predict1 = test1_id
predict1['predict_service'] = pre_test1

In [179]:
test3 = pd.read_csv('test3_featured.csv')

In [180]:
predict3 = test3[['user_id', 'predict_service']]

特征：online_time、online_mix、some_total_fee 0.73

In [226]:
predict4 = pd.read_csv('predict4_xgb.csv')
predict1 = pd.read_csv('predict1_xgb.csv')

In [227]:
predict = pd.concat([predict1, predict3, predict4])
predict.to_csv('xgb925.csv', index=None)

加上同学给的特征，分数增加了0.006

In [228]:
predict['predict_service'].value_counts()

90063345    115981
89950167     20554
89950166     17091
90109916      9923
99999828      8807
90155946      6914
99999826      5360
99999827      4314
99999830      4038
89950168      3762
99999825      3237
99104722        19
Name: predict_service, dtype: int64

In [22]:
def score(tr, te):
    labels = tr['current_service'].unique()
    f1_score = 0
    for label in labels:
        tp = 0
        fp = 0
        fn = 0
        sub_tr = tr[tr['current_service'] == label]['current_service']
        index_tr = tr[tr['current_service'] == label].index.tolist()
        sub_te = te['predict_service'][index_tr]
        for la in sub_te.values:
            if la == label:
                tp = tp + 1
            else:
                fn = fn + 1
        index_te = te[te['predict_service'] == label].index.tolist()
        fp_list = set(index_te).difference(index_tr)
        fp = len(fp_list)
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = f1_score + (2*precision*recall)/(precision+recall)
        print((2*precision*recall)/(precision+recall), label)
    final_score = (1/(len(labels))*f1_score)**2
    
    return final_score

In [119]:
from sklearn.metrics import f1_score
import numpy as np

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali['current_service'].unique()
#     preds = np.argmax(preds.reshape(8, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True